#📌 Extracción

In [24]:
import pandas as pd

# URL de los datos en formato JSON
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"

# Cargar los datos en un DataFrame
datos = pd.read_json(url)

# Mostrar las primeras filas del DataFrame para verificar la carga
datos.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


#🔧 Transformación

### Conociendo el Conjunto de Datos

Ahora que los datos han sido extraídos y cargados en un DataFrame, el siguiente paso es familiarizarnos con su estructura. Esto implica entender qué información contiene cada columna, los tipos de datos presentes y la forma general del dataset. Esta exploración inicial es crucial para planificar los pasos de limpieza y transformación.

#### Exploración de Columnas y Tipos de Datos

Vamos a utilizar el método `.info()` de Pandas para obtener un resumen técnico de nuestro DataFrame. `.info()` nos dará una visión general de las columnas, la cantidad de valores no nulos y el uso de memoria.

In [25]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   object
 1   Churn       7267 non-null   object
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(6)
memory usage: 340.8+ KB


#### Primeras Observaciones

De la salida anterior, notamos algo muy importante: las columnas `customer`, `phone`, `internet`, y `account` son de tipo `object`. Al observar las primeras filas con `.head()`, vimos que contienen datos anidados, similares a un diccionario o JSON. 

Esto significa que una parte crucial de la fase de **Transformación** será "desanidar" o "normalizar" estas columnas para que cada pieza de información (como el género del cliente, si tiene servicio de teléfono, el tipo de contrato, etc.) se convierta en su propia columna en el DataFrame. La columna `Churn` parece ser nuestro objetivo principal para el análisis, indicando si un cliente ha cancelado el servicio o no.

#### Contenido del Diccionario de Datos

Para facilitar la consulta, a continuación se muestra el contenido del diccionario de datos:

```
- `customerID`: número de identificación único de cada cliente
- `Churn`: si el cliente dejó o no la empresa
- `gender`: género (masculino y femenino)
- `SeniorCitizen`: información sobre si un cliente tiene o no una edad igual o mayor a 65 años
- `Partner`: si el cliente tiene o no una pareja
- `Dependents`: si el cliente tiene o no dependientes
- `tenure`: meses de contrato del cliente
- `PhoneService`: suscripción al servicio telefónico
- `MultipleLines`: suscripción a más de una línea telefónica
- `InternetService`: suscripción a un proveedor de internet
- `OnlineSecurity`: suscripción adicional de seguridad en línea
- `OnlineBackup`: suscripción adicional de respaldo en línea
- `DeviceProtection`: suscripción adicional de protección del dispositivo
- `TechSupport`: suscripción adicional de soporte técnico, menor tiempo de espera
- `StreamingTV`: suscripción de televisión por cable
- `StreamingMovies`: suscripción de streaming de películas
- `Contract`: tipo de contrato
- `PaperlessBilling`: si el cliente prefiere recibir la factura en línea
- `PaymentMethod`: forma de pago
- `Charges.Monthly`: total de todos los servicios del cliente por mes
- `Charges.Total`: total gastado por el cliente
```

#### Inspección de las Columnas Anidadas

Como se observó, cuatro columnas (`customer`, `phone`, `internet`, `account`) contienen datos anidados. Antes de transformarlos, es crucial inspeccionar su estructura para entender los campos que contienen. Analizaremos el primer registro de cada una como muestra.

In [26]:
datos["customer"][0]

{'gender': 'Female',
 'SeniorCitizen': 0,
 'Partner': 'Yes',
 'Dependents': 'Yes',
 'tenure': 9}

In [27]:
datos["phone"][0]

{'PhoneService': 'Yes', 'MultipleLines': 'No'}

In [28]:
datos["internet"][0]

{'InternetService': 'DSL',
 'OnlineSecurity': 'No',
 'OnlineBackup': 'Yes',
 'DeviceProtection': 'No',
 'TechSupport': 'Yes',
 'StreamingTV': 'Yes',
 'StreamingMovies': 'No'}

In [29]:
datos["account"][0]

{'Contract': 'One year',
 'PaperlessBilling': 'Yes',
 'PaymentMethod': 'Mailed check',
 'Charges': {'Monthly': 65.6, 'Total': '593.3'}}

#### Conclusiones de la Inspección

Esta inspección confirma que estas columnas son efectivamente diccionarios que deberán ser expandidos. Las claves de estos diccionarios (`gender`, `SeniorCitizen`, `Contract`, `PaymentMethod`, etc.) se convertirán en las nuevas columnas de nuestro DataFrame.

Todas estas variables son candidatas potenciales para analizar la evasión de clientes (`Churn`). En la fase de **Transformación**, nuestro primer paso será normalizar estos datos para poder utilizarlos en el análisis.

### 1. Normalización de Datos Anidados

Como primer paso en la transformación, vamos a "aplanar" o "normalizar" las columnas que contienen datos anidados (`customer`, `phone`, `internet`, `account`). Usaremos `pd.json_normalize` para expandir estas columnas y luego las uniremos en un único DataFrame limpio. Esto es esencial para poder analizar cada característica de forma individual.

In [30]:
# Normalizar las columnas con datos anidados
customer_df = pd.json_normalize(datos['customer'])
phone_df = pd.json_normalize(datos['phone'])
internet_df = pd.json_normalize(datos['internet'])

# json_normalize aplana automáticamente el diccionario 'Charges'
account_df = pd.json_normalize(datos['account'])

# Unimos todos los dataframes normalizados
datos_transformados = pd.concat([
    datos[['customerID', 'Churn']],
    customer_df,
    phone_df,
    internet_df,
    account_df
], axis=1)

print("DataFrame transformado con éxito. Nuevas dimensiones:", datos_transformados.shape)
datos_transformados.head()


DataFrame transformado con éxito. Nuevas dimensiones: (7267, 21)


,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


In [31]:
datos_transformados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


### 2. Comprobación de Incoherencias

Ahora, con el DataFrame aplanado, podemos buscar eficazmente las inconsistencias, como valores nulos, duplicados y errores de formato.

#### Búsqueda de Valores Nulos y Errores de Formato

Un problema común es que los valores faltantes no siempre están como `NaN`. A veces son cadenas vacías o espacios en blanco. La columna `Charges.Total` es un candidato para este tipo de problema, ya que es un `object` pero debería ser numérica. Vamos a investigar los registros donde `Charges.Total` parece estar vacío.

In [32]:
# Buscamos registros donde 'Charges.Total' sea un espacio en blanco
datos_transformados[datos_transformados['Charges.Total'] == ' ' ]

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
975,1371-DWPAZ,No,Female,0,Yes,Yes,0,No,No phone service,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,
1775,2520-SGTTA,No,Female,0,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,
1955,2775-SEFEE,No,Male,0,No,Yes,0,Yes,Yes,DSL,...,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,
2075,2923-ARZLG,No,Male,0,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,
2232,3115-CZMZD,No,Male,0,No,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,
2308,3213-VVOLG,No,Male,0,Yes,Yes,0,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,
2930,4075-WKNIU,No,Female,0,Yes,Yes,0,Yes,Yes,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,
3134,4367-NUYAO,No,Male,0,Yes,Yes,0,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,
3203,4472-LVYGI,No,Female,0,Yes,Yes,0,No,No phone service,DSL,...,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,
4169,5709-LVOEQ,No,Female,0,Yes,Yes,0,Yes,No,DSL,...,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,


#### Búsqueda de Duplicados

Verificaremos si existen filas completamente duplicadas en nuestro conjunto de datos.

In [33]:
print(f"Número de filas duplicadas: {datos_transformados.duplicated().sum()}")

Número de filas duplicadas: 0


#### Verificación de Valores Únicos en Columnas Categóricas

Usaremos el método `.unique()` para explorar los valores de las columnas categóricas clave y asegurarnos de que no haya valores inesperados (por ejemplo, 'Sí' y 'Si' para la misma categoría).

In [34]:
print('Valores únicos en Churn:', datos_transformados['Churn'].unique())
print('Valores únicos en gender:', datos_transformados['gender'].unique())
print('Valores únicos en Contract:', datos_transformados['Contract'].unique())
print('Valores únicos en PaymentMethod:', datos_transformados['PaymentMethod'].unique())
print('Valores únicos en InternetService:', datos_transformados['InternetService'].unique())

Valores únicos en Churn: ['No' 'Yes' '']
Valores únicos en gender: ['Female' 'Male']
Valores únicos en Contract: ['One year' 'Month-to-month' 'Two year']
Valores únicos en PaymentMethod: ['Mailed check' 'Electronic check' 'Credit card (automatic)'
 'Bank transfer (automatic)']
Valores únicos en InternetService: ['DSL' 'Fiber optic' 'No']


#### Resumen de Inconsistencias Encontradas

1.  **Datos Anidados**: Resuelto mediante la normalización del DataFrame.
2.  **Valores Faltantes/Incorrectos**: Se identificaron 11 filas donde `Charges.Total` es una cadena de espacio en blanco (' '). Esto impide convertir la columna a un tipo numérico y debe ser corregido.
3.  **Duplicados**: No se encontraron filas duplicadas.
4.  **Valores Categóricos**: Las columnas categóricas principales (`Churn`, `gender`, `Contract`, etc.) parecen tener valores consistentes y no muestran errores obvios de formato o duplicación semántica.

El siguiente paso será manejar las inconsistencias encontradas en la columna `Charges.Total`.

#📊 Carga y análisis

#📄Informe final